In [4]:
import pandas as pd
import numpy as np

In [ ]:
# 정제하자
info = pd.read_csv("all_store_info_2-1.csv")
print(info.shape)

In [ ]:
# 내가 손으로 삭제한 게 얼마나 있나
# name 없는 행 찾기 -> 452개
missing_name_rows = info[info['name'].isna()]
print(missing_name_rows)
# 삭제
info = info.dropna(subset=['name'])
print(info.shape)

In [8]:
# address 없는 행 확인
# address가 NaN이거나 빈 문자열인 행 찾기
# missing_address_rows = info[info['address'].isna()]
# print(missing_address_rows)
# address가 NaN인 행 삭제
# info = info.dropna(subset=['address'])
# print(info.shape)
# csv 저장
info.to_csv("all_store_info_2-1.csv", index=False)

In [ ]:
# 다시 읽기
info = pd.read_csv("all_store_info_2-1.csv")
print(info.shape)

In [ ]:
# address가 정확히 "서울특별시"인 행 찾기 -> 70개
seoul_rows = info[info['address'] == "서울특별시"]
print(len(seoul_rows))
# print(seoul_rows)

# seoul_rows의 id에 해당하는 naver 도메인의 name을 찾기 -> 당연히 70개
naver_names = info[(info['domain'] == 'naver') & (info['id'].isin(seoul_rows['id']))][['id', 'name']]
print(len(naver_names))
# print(naver_names)

# naver_names와 원본 데이터를 병합하여 name이 다른 행 찾기 -> 24개
merged_info = seoul_rows.merge(naver_names, on='id', suffixes=('', '_naver'))
filtered_info = merged_info[merged_info['name'] != merged_info['name_naver']]
print(len(filtered_info))
# print(filtered_info.to_string())

# 24개 중 이름이 약간 달라도 같은 집인 것 같으면 address "서울특별시 종로구"로 바꾸기
# 바꿀 id 리스트
ids_to_change = [996, 1131, 1376, 1437, 1596, 2421, 2837, 2921, 4866, 5092, 5196, 5212, 5265, 5356, 5439, 5521, 6002, 6057]
# id에 해당하는 행의 google address 값을 "서울특별시 종로구"로 변경
info.loc[(info['id'].isin(ids_to_change)) & (info['domain'] == 'google'), 'address'] = "서울특별시 종로구"

# "종로구"가 포함되지 않은 행 찾기
filtered_info = info[~info['address'].str.contains("종로구", na=False)]
print(len(filtered_info))
# 데이터 전체 출력
# print(filtered_info.to_string())

# "종로구" 있는 행만 남기기
info = info[info['address'].str.contains("종로구", na=False)]  # 조건에 맞지 않는 행 제거
print(info.shape)

In [ ]:
# csv 저장
# info.to_csv("all_store_info_2.csv", index=False)
# 다시 읽기
info = pd.read_csv("all_store_info_2-3.csv")
print(info.shape)
print(len(info["id"].unique()))

In [ ]:
# 이제 id에 한 행으로 통합해보자
# naver 데이터만 추출 (name, main_category, category, address, business_hours 가져오기)
naver_data = info[info['domain'] == 'naver']

# kakao 데이터 (business_hours가 naver에서 없을 때 사용)
kakao_data = info[info['domain'] == 'kakao']

# google 데이터 (price_per_one을 가져오기 위해 사용)
google_data = info[info['domain'] == 'google']

# rating의 평균을 계산
rating_avg = info.groupby('id')['rating'].mean().reset_index()
rating_avg['rating'] = rating_avg['rating'].round(1)

# business_hours는 naver에서 가져오되, 없으면 kakao에서 가져오기
naver_data['business_hours'] = naver_data['business_hours'].fillna(kakao_data.set_index('id')['business_hours'])

# id별로 데이터를 병합하여 최종 통합 데이터 생성
final_data = naver_data[['id', 'name', 'main_category', 'category', 'address', 'business_hours']].copy()

# rating 추가
final_data = final_data.merge(rating_avg[['id', 'rating']], on='id', how='left')

# price_per_one은 google에서 가져오기
final_data = final_data.merge(google_data[['id', 'price_per_one']], on='id', how='left')

# id를 정수형으로 변환
final_data['id'] = final_data['id'].astype(int)

# 결과 출력
print(final_data.shape)

In [ ]:
final_data

In [ ]:
print(len(final_data["name"].unique()))

# 중복된 name 찾기
duplicate_names = final_data[final_data.duplicated(subset=['name'], keep=False)]
# name을 기준으로 정렬
duplicate_names_sorted = duplicate_names.sort_values(by='name')
print(duplicate_names_sorted.to_string())

In [24]:
# csv 저장
final_data.to_csv("all_store_info_5.csv", index=False)

In [ ]:
# 리뷰~
info = pd.read_csv("all_store_info_2-1.csv")
reviews = pd.read_csv("all_store_reviews_2.csv")
print(info.shape)
print(len(info['id'].unique()))
print(reviews.shape)
print(len(reviews['id'].unique()))

In [ ]:
# (id, domain) 쌍을 집합으로 변환
info_id_domain = set(zip(info['id'], info['domain']))
reviews_id_domain = set(zip(reviews['id'], reviews['domain']))

# 합집합 (모든 (id, domain) 포함)
id_domain_union = info_id_domain | reviews_id_domain

# 교집합 (두 데이터프레임에 공통으로 존재하는 (id, domain))
id_domain_intersection = info_id_domain & reviews_id_domain

# 차집합 (info에는 있지만 reviews에는 없는 (id, domain))
id_domain_diff_info = info_id_domain - reviews_id_domain

# 차집합 (reviews에는 있지만 info에는 없는 (id, domain))
id_domain_diff_reviews = reviews_id_domain - info_id_domain

# 결과 출력
print(f"합집합 개수: {len(id_domain_union)}")
print(f"교집합 개수: {len(id_domain_intersection)}")
print(f"info에만 있는 (id, domain) 개수: {len(id_domain_diff_info)}")
print(f"reviews에만 있는 (id, domain) 개수: {len(id_domain_diff_reviews)}")

# 실제 값 출력 (필요하면 주석 해제)
# print("info에만 있는 (id, domain) 목록:", id_domain_diff_info)
# print("reviews에만 있는 (id, domain) 목록:", id_domain_diff_reviews)

In [ ]:
# 교집합 (id, domain) 기준으로 reviews 데이터 필터링
filtered_reviews = reviews[reviews.set_index(['id', 'domain']).index.isin(id_domain_intersection)]

# 결과 출력
print(filtered_reviews.shape)
print(filtered_reviews.head())

In [ ]:
# (id, domain) 쌍을 집합으로 변환
info_id_domain = set(zip(info['id'], info['domain']))
reviews_id_domain = set(zip(filtered_reviews['id'], filtered_reviews['domain']))

# 합집합 (모든 (id, domain) 포함)
id_domain_union = info_id_domain | reviews_id_domain

# 교집합 (두 데이터프레임에 공통으로 존재하는 (id, domain))
id_domain_intersection = info_id_domain & reviews_id_domain

# 차집합 (info에는 있지만 reviews에는 없는 (id, domain))
id_domain_diff_info = info_id_domain - reviews_id_domain

# 차집합 (reviews에는 있지만 info에는 없는 (id, domain))
id_domain_diff_reviews = reviews_id_domain - info_id_domain

# 결과 출력
print(f"합집합 개수: {len(id_domain_union)}")
print(f"교집합 개수: {len(id_domain_intersection)}")
print(f"info에만 있는 (id, domain) 개수: {len(id_domain_diff_info)}")
print(f"reviews에만 있는 (id, domain) 개수: {len(id_domain_diff_reviews)}")

# 실제 값 출력 (필요하면 주석 해제)
# print("info에만 있는 (id, domain) 목록:", id_domain_diff_info)
# print("reviews에만 있는 (id, domain) 목록:", id_domain_diff_reviews)

In [ ]:
# info와 reviews의 id 집합 생성
info_ids = set(info['id'])
reviews_ids = set(reviews['id'])

# 합집합 (모든 id 포함, 중복 제거)
id_union = info_ids | reviews_ids
# 교집합 (둘 다 포함된 id)
id_intersection = info_ids & reviews_ids
# 차집합 (info에는 있지만 reviews에는 없는 id)
id_diff_info = info_ids - reviews_ids
# 차집합 (reviews에는 있지만 info에는 없는 id)
id_diff_reviews = reviews_ids - info_ids

# 결과 출력
print(f"합집합 개수: {len(id_union)}")
print(f"교집합 개수: {len(id_intersection)}")
print(f"info에만 있는 id 개수: {len(id_diff_info)}")
print(f"reviews에만 있는 id 개수: {len(id_diff_reviews)}")

# 실제 값 출력 (필요하면 주석 해제)
# print("info에만 있는 id 목록:", id_diff_info)
# print("reviews에만 있는 id 목록:", id_diff_reviews)

In [ ]:
print(len(info['id'].unique()))
print(len(filtered_reviews['id'].unique()))

In [ ]:
missing_ids = set(info['id'].unique()) - set(filtered_reviews['id'].unique())
print(f"info에는 있지만 reviews에는 없는 id 개수: {len(missing_ids)}")
print("일부 missing id 예시:", list(missing_ids)[:10])

In [45]:
# csv 저장
filtered_reviews.to_csv("all_store_reviews_3.csv", index=False)

In [ ]:
# 리뷰 정제
reviews = pd.read_csv("all_store_reviews_3.csv")
print(reviews.shape)
print(len(reviews['id'].unique()))
missing_reviews = reviews[reviews['reviews'].isna() | (reviews['reviews'].str.strip() == '')]
print(missing_reviews)

In [ ]:
import emoji

# 이모티콘 제거 함수
def remove_emojis(text):
    return emoji.replace_emoji(text, "")

# 'reviews' 열에서 이모티콘만 제거
reviews['reviews'] = reviews['reviews'].apply(remove_emojis)

# 이모티콘 제거로 인한 리뷰 공백 확인 후 제거 -> 472개 제거 (161652개 남음)
missing_reviews = reviews[reviews['reviews'].isna() | (reviews['reviews'].str.strip() == '')]
print(missing_reviews.shape)
reviews_cleaned = reviews.drop(missing_reviews.index)
reviews_cleaned = reviews_cleaned.reset_index(drop=True)
print(reviews_cleaned.shape)
print(reviews.shape)

In [ ]:
import re

def clean_text(text):
    # 줄바꿈을 공백으로 바꿔줌
    text = re.sub(r'\n+', ' ', text)
    
    return text

# 'reviews' 열에서 줄바꿈 처리
reviews_cleaned2 = reviews_cleaned.copy()
reviews_cleaned2['reviews'] = reviews_cleaned2['reviews'].apply(clean_text)

# 결과 확인
print(reviews_cleaned2.head())

In [ ]:
# 숫자로만 이루어진 리뷰 찾기 -> 13개
numeric_reviews = reviews_cleaned2[reviews_cleaned2['reviews'].str.match(r'^\d+(\.\d+)?$', na=False)]
print(numeric_reviews.shape)
# print(numeric_reviews)

# 특수문자로만 이루어진 리뷰 찾기 -> 549개
special_char_reviews = reviews_cleaned2[reviews_cleaned2['reviews'].str.match(r'^[^\w\s]+$', na=False)]
print(special_char_reviews.shape)
# print(special_char_reviews.to_string())

# 위의 리뷰 제외
reviews_cleaned3 = reviews_cleaned2.copy()
reviews_cleaned3 = reviews_cleaned3[~reviews_cleaned3['reviews'].str.match(r'^\d+(\.\d+)?$', na=False)]
reviews_cleaned3 = reviews_cleaned3[~reviews_cleaned3['reviews'].str.match(r'^[^\w\s]+$', na=False)]
reviews_cleaned3 = reviews_cleaned3.reset_index(drop=True)
print(reviews_cleaned3.shape)

In [ ]:
# # 자음과 모음만 있는 부분을 찾아 출력하는 함수
# def find_consonant_vowel(text):
#     # 자음만, 모음만, 자음+모음 혼합 포함된 부분 찾기
#     pattern = r'[ㄱ-ㅎㅏ-ㅣ]+'  # 자음만 또는 모음만 있는 부분을 찾는 정규식
#     matches = re.findall(pattern, text)
#     return matches

# # 'reviews' 열에서 자음과 모음만 있는 부분 찾기
reviews_cleaned4 = reviews_cleaned3.copy()  # 원본 복사
# reviews_cleaned4['consonant_vowel_only'] = reviews_cleaned4['reviews'].apply(find_consonant_vowel)

# # 자음과 모음만 포함된 부분 출력
# matches_reviews = reviews_cleaned4[reviews_cleaned4['consonant_vowel_only'].apply(lambda x: len(x) > 0)]
# # print(matches_reviews[['reviews', 'consonant_vowel_only']].to_string())

# 자음, 모음만 있는 경우를 제거하는 정규 표현식
def remove_consonant_vowel(text):
    # 자음만, 모음만, 자음+모음 혼합 포함된 부분을 찾아 제거
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', '', text)  # 자음만 또는 모음만 있는 부분을 제거
    text = re.sub(r'\s+', ' ', text)  # 여러 공백을 하나의 공백으로 치환
    text = text.strip()  # 앞뒤 공백 제거
    return text

# 'reviews' 열에서 자음이나 모음만 있는 경우 제거
reviews_cleaned4['reviews'] = reviews_cleaned4['reviews'].apply(remove_consonant_vowel)

# 결과 확인
print(reviews_cleaned4.shape)
print(reviews_cleaned4.to_string())

In [ ]:
# 특수문자 제외하고 5글자 이하인 리뷰 찾기

# 특수문자 제거 함수
def remove_special_chars(text):
    # 특수문자 제거 (영어, 한글, 숫자, 공백은 남김)
    text = re.sub(r'[^\w\s]', '', text)
    # 공백 처리
    text = ' '.join(text.split())
    return text

# 특수문자 제거 후 공백 처리
reviews_cleaned5 = reviews_cleaned4.copy() #.drop(columns=['consonant_vowel_only'])
reviews_cleaned5['reviews'] = reviews_cleaned5['reviews'].apply(remove_special_chars)
# print(reviews_cleaned5.to_string())

# # 길이가 5글자 이하인 값 찾기
# filtered_reviews = reviews_cleaned5[reviews_cleaned5['reviews'].str.len() <= 5]
# print(filtered_reviews.shape)
# print(filtered_reviews.to_string())

# 길이가 5글자 이하인 리뷰가 있는 행 삭제
reviews_cleaned6 = reviews_cleaned5[reviews_cleaned5['reviews'].str.len() > 5]
print(reviews_cleaned6.shape)

In [14]:
# csv 저장
reviews_cleaned6.to_csv("all_store_reviews_4.csv", index=False)

In [ ]:
# 다시 읽기
info = pd.read_csv("all_store_info_2-1.csv")
reviews = pd.read_csv("all_store_reviews_4.csv")
print(info.shape)
print(reviews.shape)
print(len(info['id'].unique()))
print(len(reviews['id'].unique()))

In [ ]:
# info 에서 없어진 리뷰 해당 id 삭제
info_filtered = info[info['id'].isin(reviews['id'])]
print(info_filtered.shape)
print(len(info_filtered['id'].unique()))

In [ ]:
# 'id' 컬럼을 정수형으로 변환
info_filtered['id'] = info_filtered['id'].astype(int)
info_filtered

In [19]:
# csv 저장
info_filtered.to_csv("all_store_info_2-3.csv", index=False)

In [ ]:
# 마지막 확인
info = pd.read_csv("all_store_info_5.csv")
reviews = pd.read_csv("all_store_reviews_4.csv")
print(info.shape)
print(reviews.shape)
print(len(info['id'].unique()))
print(len(reviews['id'].unique()))

In [ ]:
# info와 reviews의 id 집합 생성
info_ids = set(info['id'])
reviews_ids = set(reviews['id'])

# 합집합 (모든 id 포함, 중복 제거)
id_union = info_ids | reviews_ids
# 교집합 (둘 다 포함된 id)
id_intersection = info_ids & reviews_ids
# 차집합 (info에는 있지만 reviews에는 없는 id)
id_diff_info = info_ids - reviews_ids
# 차집합 (reviews에는 있지만 info에는 없는 id)
id_diff_reviews = reviews_ids - info_ids

# 결과 출력
print(f"합집합 개수: {len(id_union)}")
print(f"교집합 개수: {len(id_intersection)}")
print(f"info에만 있는 id 개수: {len(id_diff_info)}")
print(f"reviews에만 있는 id 개수: {len(id_diff_reviews)}")

# 실제 값 출력 (필요하면 주석 해제)
# print("info에만 있는 id 목록:", id_diff_info)
# print("reviews에만 있는 id 목록:", id_diff_reviews)